In [56]:
import numpy as np
import mdtraj as md
import matplotlib.pyplot as plt
import pyemma
from pyemma import msm, plots    # pyemma APIs

import scipy
import scipy.sparse as sps  

import time
import scipy.sparse.linalg as spl
import sklearn.neighbors as neigh_search      
import sklearn.cluster as skl_cl
import sys

%pylab inline
%config InlineBackend.figure_format='svg'

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


## Space Time Diffusion Map helper functions

In [41]:
def kernel_neighbor_search(A, r, epsilon, sparse=False):
    """
    Analyzes one frame: uses a nearest neighbor algorithm to compute all distances up to cutoff r, 
    generates the diffusion kernel sparse matrix
    
    Parameters:
        A:   nparray (m, 3), m number of heavy atoms
             array of coordinates of heavy atoms
        r:   scalar, cutoff
        epsilon: scalar, localscale
             
    Return:
        kernel: sparse matrix (m,m)
                diffusion kernel matrix (to be passed on to the SpaceTimeDMap subroutine)
    """

    #calling nearest neighbor search class
    kernel = neigh_search.radius_neighbors_graph(A, r, mode='distance')
    # computing the diffusion kernel value at the non zero matrix entries
    kernel.data = np.exp(-(kernel.data**2)/(epsilon))

    # diagonal needs to be added separately
    kernel = kernel + sps.identity(kernel.shape[0], format = 'csr')

    if sparse:
        return kernel
    else:
        return kernel.toarray()



In [42]:
def matrix_B(kernel, sparse = False):

    if sparse:
        m = kernel.shape[0]
        D = sps.csr_matrix.sum(kernel, axis=0)
        Q = sps.spdiags(1./D, 0, m, m)
        S = kernel * Q
        B = (S*(sps.csr_matrix.transpose(S)))/(sps.csr_matrix.sum(S, axis=1))
    else:
        D = np.sum(kernel, axis = 1)
        S = kernel*(1./D)
        B = (np.dot(S, S.T)/(np.sum(S, axis = 1))).T

    return B

In [69]:
def compute_SpaceTimeDMap(X, r, epsilon, sparse=False):
    """
    computes the SpaceTime DIffusion Map matrix out of the dataset available
    Parameters
    -------------------
    X: array T x m x 3
      array of T time slices x 3m features (m=number of atoms, features are xyz-coordinates of the atoms)
    r: scalar
      cutoff radius for diffusion kernel
    epsilon: scalar
      scale parameter
    
    Returns
    -------------------
    ll: np.darray(m)
      eigenvalues of the SpaceTime DMap
    u: ndarray(m,m)
      eigenvectors of the SpaceTime DMap. u[:,i] is the ith eigenvector corresponding to i-th eigenvalue
    SptDM: ndarray(m,m)
      SpaceTime Diffusion Matrix, eq (3.13) in the paper, time average of all the matrices in the cell list
    """

    # initialize the Spacetime Diffusion Map matrix 
    # that will be averaged over the different timeslices 
    # and the over the different trajectories 
    m = np.shape(X)[1]
    T = np.shape(X)[0]
    
    #SptDM =  sps.csr_matrix((m, m)) 
    SptDM = np.zeros((m,m))

    # loop over trajectory
    
    for i_t in range(T):
        if (i_t % 1e4==0):
            print 'time slice ' + str(i_t)
        # selecting the heavy atoms coordinates in the timeslice s
        # compute diffusion kernel using data at timeslice s
        distance_kernel = kernel_neighbor_search(X[i_t,:,:], r, epsilon, sparse=sparse)
        SptDM += matrix_B(distance_kernel, sparse=sparse)

    # divide by the total number of timeslices considered
    # this define the Q operator 
    SptDM /= T

    # Computing eigenvalues and eigenvectors of the SpaceTime DMap
    if sparse:
        ll, u = spl.eigs(SptDM, k = 50, which = 'LR')
        ll, u = sort_by_norm(ll, u)
    else:
        ll, u = np.linalg.eig(SptDM)
        ll, u = sort_by_norm(ll, u)

    return ll, u, SptDM

In [70]:
def sort_by_norm(evals, evecs):
    """
    Sorts the eigenvalues and eigenvectors by descending norm of the eigenvalues
    Parameters
    ----------
    evals: ndarray(n)
        eigenvalues
    evecs: ndarray(n,n)
        eigenvectors in a column matrix
    Returns
    -------
    (evals, evecs) : ndarray(m), ndarray(n,m)
        the sorted eigenvalues and eigenvectors
    """
    # norms
    evnorms = np.abs(evals)
    # sort
    I = np.argsort(evnorms)[::-1]
    # permute
    evals2 = evals[I]
    evecs2 = evecs[:, I]
    # done
    return (evals2, evecs2)

In [71]:
def reshape_array(X):
    T, m = np.shape(X)
    d = 3
    m = m / d
    A = np.zeros((T, m, d))
    for i_t in range(T):
        A[i_t,:,0] = X[i_t,np.arange(0,3*m,3)];
        A[i_t,:,1] = X[i_t,np.arange(1,3*m,3)];
        A[i_t,:,2] = X[i_t,np.arange(2,3*m,3)];
    return A

### parameters

In [62]:
delta = 0.2   # in ns
print('Trajectory timestep [ns]   = ' + str(delta))

# topology file
topfile = path + 'firstframe.pdb'

# trajectory file
filename = path + 'ww_1-protein-00' + str(i) + '.dcd'

# indices of those configurations to run SpaceTime DMap on
my_idx = np.random.randint(2000, size = 50)
print('Number of configurations   = ' + str(len(my_idx)))

eps_list = [0.4, 0.6, 0.8]
print('Considering epsilon values = ' + str(eps_list))


out_file = 'fip35_example.npz'
print('Output file                = ' + str(out_file))

path = './'

Number of trajectories     = 2
Trajectory timestep [ns]   = 0.2
Number of configurations   = 50
Considering epsilon values = [0.4, 0.6, 0.8]
Output file                = fip35_example.npz


### Loading trajectory data

In [75]:
# load an look into topology file
reftraj = md.load(topfile)
print reftraj
print reftraj.topology
print '\n'

print('Initializing name of trajectory file:')
print(filename)
sys.stdout.flush()

<mdtraj.Trajectory with 1 frames, 562 atoms, 35 residues, without unitcells>
<mdtraj.Topology with 1 chains, 35 residues, 562 atoms, 571 bonds>


Initializing name of trajectory file:
./ww_1-protein-001.dcd


In [72]:
import pyemma.coordinates as coor

# define features to load for spacetime diffusion map analysis

# use heavy atom coordinates is the default option. Other option (such as contacts) will be available in
# a future release

#------------------------
# loading the trajectory will require (most likely) a lot of computing time and a lot of memory.
# Should this be case, we strongly recommend that the whole calculation be run on a cluster and not 
# on one's own laptop
#------------------------

print('define basis functions: heavy atom coordinates')
sys.stdout.flush()

featurizer = coor.featurizer(topfile)
featurizer.add_selection(featurizer.select_Heavy())

#print(featurizer.describe())        # extensive list of features used
#print(featurizer.dimension())        # number of features input
#sys.stdout.flush()

# use featurizer to read in trajectory
print('Loading trajectory...')
inp = coor.source(filename, featurizer, chunk_size=10000)

# extract output into an array
X = np.array(inp.get_output()[0])
print X.shape

# extracting the (indices) subset of configurations from the whole trajectory that was just loaded
print('\n Subsampling trajectory...')
X_slice = X[my_idx,:]
print('number of frames to use as input below = ' + str(np.shape(X_slice)[0]))

print('\n')
if np.shape(X_slice)[0] != len(my_idx):
    print('ACHTUNG! Something went wrong with the selection of the frames')

define basis functions: heavy atom coordinates
Loading trajectory...
(10000, 864)

 Subsampling trajectory...
number of frames to use as input below = 50




### Load Space Time Diffusion Map helper functions

In [73]:
#--------------------------------------------------------------------
# Effectively Run spacetime diffusion map analysis
# --------------------------------------------------------------------

print(' ################         Perform spacetime diffusion map calculations...     ################')

print('Reshaping...')
X = reshape_array(X_slice)
print X.shape
sys.stdout.flush()

n_conf = len(my_idx)      # number of configurations

matrix = []               # space time diffusion map matrix
eigenvalues = []          # eigenvalues
eigenvectors = []         # eigenvectors (--> to be used as clustering coordinates later)


for eps in eps_list:
    
    print('epsilon = ' + str(eps))
    sys.stdout.flush()
    r = 2*np.sqrt(eps);
    
    ll, u, SptDM = compute_SpaceTimeDMap(X, r, eps, sparse=False);  
    
    # append results
    eigenvalues.append(ll)
    eigenvectors.append(u)
    matrix.append(SptDM)

 ################         Perform spacetime diffusion map calculations...     ################
Reshaping...
(50, 288, 3)
epsilon = 0.4
time slice 0
epsilon = 0.6
time slice 0
epsilon = 0.8
time slice 0


In [74]:
#----------------------------------------------------------------------
# save results
#---------------------------------------------------------------------

# saving step
my_dict = {}

my_dict['epsilon'] = eps_list
my_dict['lambdas'] = eigenvalues
my_dict['eigenvectors'] = eigenvectors
my_dict['matrix']  = matrix
my_dict['number_of_frames'] = n_conf
#my_dict['config'] = X.shape[1]

# this is the file where spacetimediffusionmap results are going to be stored
print('File where space time diffusion map results are going to be stored in = ' + str(out_file))
np.savez_compressed(out_file, **my_dict)

print('Computation and Saving step successful! Data ready to be analyzed')

File where space time diffusion map results are going to be stored in = fip35_example.npz
Computation and Saving step successful! Data ready to be analyzed
